# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](part1_pass2.PNG)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](part1_pass1.PNG)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [1]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import scipy.signal

import matplotlib.pyplot as plt
%matplotlib inline

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls


def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability.

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and
        corresponding reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse
        rate error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))    

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an
    aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return (AggregateErrorMetric(errs, confs), errs, confs)    

def BandpassFilter(signal, bandpass=(40/60,240/60), fs=125):
    """
    Bandpass Filter:
        Filers the input signal, in this case between 40 and 240 BPM

    Args:
        signal:     (np.array) The input signal
        pass_band:  (tuple) The pass band. Frequency components outside the bandpass are filered.
        fs:         (number) The sampling rate of <signal>
        
    Returns:
        (np.array) The filtered signal
    """
    b, a = sp.signal.butter(3, bandpass, btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)


def Features(signal, fs=125):
    """ 
    Features:
        A featurization of the signal data
    Args:
        signal:     (np.array) The input signal
        fs:         (number) The sampling rate of <signal>
    returns:
        list of frequencies, fft and weights
    """

    # run the descrete Fourier Transform. it returns a float array
    freqs = np.fft.rfftfreq(len(signal), 1 / fs)

    # Take an FFT of the centered signal
    fft = np.abs(np.fft.rfft(signal - np.mean(signal), len(signal)))
    # filter the range between the lower and higher BPM
    fft[freqs <= ((40+30) / 60)] = 0.0
    fft[freqs >= ((240-70) / 60)] = 0.0

    # the weight of the signal
    weight = np.sum(np.square(signal - np.mean(signal)))

    return freqs, fft, weight

def EstimateMaxPPG(ppg_weight, acc_weight, ppg_frequencies, acc_frequencies, ppg_fft, acc_fft):
    """
    EstmateMaxPPG
       The function return a max PPG and max acc value which is used the calculate the BPM
    Args:
        ppg_weight:         (float) a weight number calculated on the ppg signal 
        acc_weight:         (float) a weight number calculated on the acc signal
        ppg_frequencies:    (array) ppg signal
        acc_frequencies:    (array) acc signal
        ppg_fft:            (array) ppg tuple
        acc_fft:            (array) accelorometer tuple
    returns:
        (float) with a max ppg and max acc 
    """
    max_ppg = ppg_frequencies[np.argmax(ppg_weight)]
    max_acc = acc_frequencies[np.argmax(acc_weight)]

    lcv = 1       
    while np.abs(max_ppg - max_acc) <= 0 and lcv <=2:
        lcv += 1
        max_ppg = ppg_frequencies[np.argsort(ppg_fft, axis = 0)[-lcv]]
        max_acc = acc_frequencies[np.argsort(acc_fft, axis = 0)[-lcv]]
    
    return max_ppg, max_acc

def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    RunPulseRateAlgorithm:
        Main part of the program. it estimates the pulse rate and 
        returns an error and confidence
    Args:
        data_fl:    (string) path to the TROIKA data file (.mat)
        ref_fl:     (string) path to the reference file (ground truth)
    Returns:
        error rate between the estimation and reference
        a confidence metric of the resulted BPM    
    """
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    
    # Load the reference data, also named as "ground truth" heart rate data
    reference_bpm = sp.io.loadmat(ref_fl)['BPM0']
    
    # filter signals
    ppg  = BandpassFilter(ppg)
    accx = BandpassFilter(accx)
    accy = BandpassFilter(accy)
    accz = BandpassFilter(accz)
    
    # Calculate the acc magnitude
    acc_magnitude = np.sqrt(np.square(accx) + np.square(accy) + np.square(accz))
    
    estimates = []
    confidences = []
    fs = 125
    window_length = 8 * fs
    window_shift = 2 * fs
   
    for i in range(0, len(ppg) - window_length, window_shift):
        
        ppg_freqs, ppg_fft, ppg_weigth = Features(ppg[i:i + window_length], fs)
        acc_freqs, acc_fft, acc_weigth = Features(acc_magnitude[i:i + window_length], fs)

        est_freqs = EstimateMaxPPG(ppg_weigth, acc_weigth, ppg_freqs, acc_freqs, ppg_fft, acc_fft)[0]  
        estimates.append(est_freqs * 60)
        
        # calculate the confidence with a windows frequency of 40/60 (Hz)
        window_f = 40/60
        confidence = np.sum(ppg_fft[(ppg_freqs >= (est_freqs - window_f)) & (ppg_freqs <= (est_freqs + window_f))])/ np.sum(ppg_fft)                
        confidences.append(confidence) 
        
        # Return MEA and confidence.
    errors = np.abs(np.diag(estimates - reference_bpm))
    return np.array(errors), np.array(confidences)